In [3]:
# Copyright (2024) Bytedance Ltd. and/or its affiliates
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import json
import os

# This is an example for DREAM

def vid2path(video_dir, vid, dataset="DREAM-1K"):
    if dataset == "DREAM-1K":
        # For DREAM-1K, vid is the video filename without extension
        return f"{video_dir}/{vid}.mp4"
    if dataset == "TVBench":
        # For TVBench, vid is the video filename with sub dir
        return f"{video_dir}/{vid}"
    else:
        # For other benchmarks, you should checkout their metadata and rewrite this function
        raise NotImplementedError(f"Unknown dataset: {dataset}")
    

def fill_in_video_file(input_file, video_dir, dataset):
    data = [json.loads(line) for line in open(input_file)]
    for d in data:
        vid = str(d['vid'])
        video_file = vid2path(video_dir, vid, dataset)
        assert os.path.exists(video_file)
        for msg in d['messages']:
            if msg['role'] == "user":
                for content in msg['content']:
                    if content['type'] == 'video':
                        content['video']['video_file'] = video_file
    with open (input_file, 'w') as fout:
        for d in data:
            fout.write(json.dumps(d, ensure_ascii=False) + '\n')

In [4]:
# prepare DREAM-1K
video_dir = "DIR_TO_DREAM_VIDEOS"
input_file = "annotations/DREAM-1k.jsonl"
fill_in_video_file(input_file, video_dir, dataset="DREAM-1K")

In [4]:
# prepare TVBench
# video_dir = "DIR_TO_TVBENCH_VIDEOS"
video_dir = "/mnt/bn/writing-model-training/videos/TVBench/videos"
input_file = "annotations/TVBench.jsonl"
fill_in_video_file(input_file, video_dir, dataset="TVBench")